In [1]:
import pandas as pd


In [5]:
df_customers = pd.read_csv("..\\data\\raw\\customers.csv")
df_loans = pd.read_csv("..\\data\\raw\\loans.csv")
df_bureau = pd.read_csv("..\\data\\raw\\bureau_data.csv")

In [6]:
df_customers.shape, df_loans.shape, df_bureau.shape

((50000, 12), (50000, 15), (50000, 8))

In [7]:
df = pd.merge(df_customers, df_loans, on="cust_id")
df.shape

(50000, 26)

In [8]:
df = pd.merge(df, df_bureau)
df.shape

(50000, 33)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cust_id                      50000 non-null  object 
 1   age                          50000 non-null  int64  
 2   gender                       50000 non-null  object 
 3   marital_status               50000 non-null  object 
 4   employment_status            50000 non-null  object 
 5   income                       50000 non-null  int64  
 6   number_of_dependants         50000 non-null  int64  
 7   residence_type               49938 non-null  object 
 8   years_at_current_address     50000 non-null  int64  
 9   city                         50000 non-null  object 
 10  state                        50000 non-null  object 
 11  zipcode                      50000 non-null  object 
 12  loan_id                      50000 non-null  object 
 13  loan_purpose    

In [10]:
df["default"] = df["default"].astype(int)
df["default"].value_counts()

default
0    45703
1     4297
Name: count, dtype: int64

In [11]:
df["cust_id"].nunique() == len(df)


True

In [12]:
ids = df["cust_id"]

In [13]:
from sklearn.model_selection import train_test_split
#Split 70train, 20val,10test
train_ids, rest_ids = train_test_split(ids, test_size=0.3, random_state=42, stratify=df["default"])
val_ids, test_ids = train_test_split(rest_ids, test_size=1/3, random_state=42, stratify=df[df["cust_id"].isin(rest_ids)]["default"])


In [14]:
#save IDs
pd.Series(train_ids).to_csv("..\\data\\split\\train_ids.txt", index=False, header=False)
pd.Series(val_ids).to_csv("..\\data\\split\\val_ids.txt", index=False, header=False)
pd.Series(test_ids).to_csv("..\\data\\split\\test_ids.txt", index=False, header=False)

In [17]:
train_ids = pd.read_csv("..\\data\\split\\train_ids.txt", header=None)[0].astype(df["cust_id"].dtype)

df_train = df[df["cust_id"].isin(train_ids)].reset_index(drop=True)
print(df_train.shape)
df_train.head()


(35000, 33)


,cust_id,age,gender,marital_status,employment_status,income,number_of_dependants,residence_type,years_at_current_address,city,...,disbursal_date,installment_start_dt,default,number_of_open_accounts,number_of_closed_accounts,total_loan_months,delinquent_months,total_dpd,enquiry_count,credit_utilization_ratio
0,C00001,44,M,Married,Self-Employed,113784,3,Owned,27,Bydgoszcz,...,24.07.2019,10.08.2019,0,1,1,42,0,0,3,7
1,C00002,38,M,Married,Salaried,53064,3,Owned,4,Poznan,...,24.07.2019,15.08.2019,0,3,1,96,12,60,5,4
2,C00004,55,F,Single,Self-Employed,156068,1,Owned,15,Bydgoszcz,...,24.07.2019,09.08.2019,0,3,0,115,15,87,5,26
3,C00005,37,M,Married,Salaried,151008,3,Owned,28,Wroclaw,...,24.07.2019,02.08.2019,0,4,2,120,0,0,5,10
4,C00007,55,M,Married,Self-Employed,143220,3,Mortgage,21,Gdansk,...,24.07.2019,22.08.2019,0,2,2,79,2,16,4,12
